# Local Field Potential (LFP) Analysis Notebook

This notebook performs Local Field Potential (LFP) analysis using Python. We will cover:
1. **Data Loading**: Using Neo and SpikeInterface for loading electrophysiological data.
2. **Preprocessing**: Filtering and cleaning the data.
3. **Time-Frequency Analysis**:
   - Short-Time Fourier Transform (STFT)
   - Wavelet Transform
4. **Coherence Analysis**: Measuring coherence between LFP signals.
5. **Phase-Amplitude Coupling (PAC)**: Analyzing coupling between low-frequency phase and high-frequency amplitude.
6. **Cross-Frequency Coupling (CFC)**: Measuring interactions between oscillatory activities at different frequencies.
7. **Visualization**: Plotting power spectral densities, coherence plots, and phase-amplitude coupling indices.

We use established libraries such as **Neo**, **SpikeInterface**, **Elephant**, **SciPy**, and **PyWavelets**.

In [ ]:
# Import necessary libraries
import neo  # For data handling
import spikeinterface as si  # Core module for SpikeInterface
import spikeinterface.extractors as se  # For data loading and extraction
import spikeinterface.preprocessing as sp  # For data preprocessing
import elephant  # For advanced LFP analysis
import elephant.spectral as espt  # For spectral analysis
import elephant.coherence as ecoh  # For coherence analysis
import elephant.phase_analysis as ephase  # For PAC/CFC analysis
import scipy.signal as signal  # For STFT and spectral methods
import pywt  # For Wavelet Transform
import quantities as pq  # For unit handling
import matplotlib.pyplot as plt  # For static visualization
import plotly.graph_objects as go  # For interactive visualization
import numpy as np  # For numerical operations
from neo.io import NeuralynxIO  # Example IO for Neo data loading

# Set plot style
plt.style.use('seaborn')

# Define global parameters
fs = 1000  # Example sampling frequency, replace with your dataset's value

## Step 1: Load Data
Load electrophysiological data using Neo and convert it to SpikeInterface format.

In [ ]:
# Data Handling Module
def load_data(file_path):
    """
    Load electrophysiological data using Neo and convert to SpikeInterface format.
    
    Args:
    - file_path (str): Path to the file containing raw data.
    
    Returns:
    - recording (si.BaseRecording): Loaded data in SpikeInterface's RecordingExtractor format.
    """
    reader = NeuralynxIO(dirname=file_path)
    block = reader.read_block()
    segment = block.segments[0]
    analog_signal = segment.analogsignals[0]
    recording = se.NeoRecordingExtractor(analog_signal)
    return recording

# Example file path for demonstration
file_path = 'data/sample_lfp_data'  # Adjust the path for your dataset

# Load Data
recording = load_data(file_path)
print("Data loaded successfully.")

## Step 2: Preprocess Data
Apply bandpass filtering and notch filtering to clean the LFP signals.

In [ ]:
# Preprocessing Module
def preprocess_data(recording, freq_min=1, freq_max=100, notch_freq=None):
    """
    Preprocess the loaded data by applying bandpass filtering and optional notch filtering.
    
    Args:
    - recording (si.BaseRecording): Loaded data in SpikeInterface's RecordingExtractor format.
    - freq_min (float): Minimum frequency for bandpass filter.
    - freq_max (float): Maximum frequency for bandpass filter.
    - notch_freq (float): Frequency for notch filter to remove powerline noise. If None, skip.
    
    Returns:
    - recording_preprocessed (si.BaseRecording): Preprocessed LFP data.
    """
    # Bandpass filter for LFP
    recording_bp = sp.bandpass_filter(recording, freq_min=freq_min, freq_max=freq_max)
    
    # Optional notch filter
    if notch_freq:
        recording_notch = sp.notch_filter(recording_bp, freq=notch_freq)
        return recording_notch
    return recording_bp

# Preprocess Data
recording_preprocessed = preprocess_data(recording, freq_min=1, freq_max=100, notch_freq=50)
print("Data preprocessed successfully.")

## Step 3: Time-Frequency Analysis
We perform two types of time-frequency analysis:
1. **Short-Time Fourier Transform (STFT)**
2. **Wavelet Transform**

In [ ]:
# STFT Analysis Module
def time_frequency_analysis_stft(analog_signal, fs=1000, nperseg=256):
    """
    Perform time-frequency analysis using Short-Time Fourier Transform (STFT).
    
    Args:
    - analog_signal (neo.AnalogSignal): LFP data in Neo's AnalogSignal format.
    - fs (int): Sampling frequency.
    - nperseg (int): Length of each segment for STFT.
    
    Returns:
    - f (np.ndarray): Frequency bins.
    - t (np.ndarray): Time bins.
    - Zxx (np.ndarray): STFT result.
    """
    f, t, Zxx = signal.stft(analog_signal.magnitude.flatten(), fs=fs, nperseg=nperseg)
    return f, t, Zxx

# Perform STFT Analysis
analog_signal = recording_preprocessed.get_traces().T[0] * pq.uV  # Example channel
f, t, Zxx = time_frequency_analysis_stft(analog_signal, fs=fs)

# Plot STFT
plt.figure(figsize=(10, 6))
plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.colorbar(label='Magnitude')
plt.show()

### Wavelet Transform Analysis
Wavelet Transforms allow multi-resolution analysis of the signal.

In [ ]:
# Wavelet Analysis Module
def time_frequency_analysis_wavelet(analog_signal, wavelet='cmor', scales=np.arange(1, 128)):
    """
    Perform time-frequency analysis using Wavelet Transform.
    
    Args:
    - analog_signal (neo.AnalogSignal): LFP data in Neo's AnalogSignal format.
    - wavelet (str): Type of wavelet to use (e.g., 'cmor').
    - scales (np.ndarray): Scales to use in wavelet transform.
    
    Returns:
    - coeffs (np.ndarray): Wavelet coefficients.
    - freqs (np.ndarray): Frequencies corresponding to scales.
    """
    coeffs, freqs = pywt.cwt(analog_signal.magnitude.flatten(), scales, wavelet)
    return coeffs, freqs

# Perform Wavelet Transform
coeffs, freqs = time_frequency_analysis_wavelet(analog_signal)

# Plot Wavelet Transform
plt.figure(figsize=(10, 6))
plt.imshow(np.abs(coeffs), extent=[0, len(analog_signal) / fs, freqs[-1], freqs[0]], aspect='auto', cmap='jet')
plt.colorbar(label='Magnitude')
plt.title('Wavelet Transform')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

## Coherence Analysis

Coherence analysis quantifies the synchrony between two signals, which can provide insights into the functional connectivity between brain regions.


In [ ]:
# Python Cell: Coherence Analysis
from lfp_analysis import coherence_analysis

# Assume we are comparing the same signal for simplicity in demonstration
coherency, freqs_coherence = coherence_analysis(preprocessed_data, preprocessed_data)

# Plot Coherence
fig = px.line(x=freqs_coherence, y=coherency, labels={'x': 'Frequency (Hz)', 'y': 'Coherence'},
              title='Coherence Analysis between Signals')
fig.show()

## Step 4: Coherence Analysis
Assess the coherence between two LFP signals to understand interactions between different brain regions.


In [ ]:
# Coherence Analysis Module
def coherence_analysis(analog_signal1, analog_signal2, fs=1000):
    """
    Assess coherence between two LFP signals.
    
    Args:
    - analog_signal1 (neo.AnalogSignal): First LFP signal.
    - analog_signal2 (neo.AnalogSignal): Second LFP signal.
    - fs (int): Sampling frequency.
    
    Returns:
    - f (np.ndarray): Frequency bins.
    - Cxy (np.ndarray): Coherence values.
    """
    f, Cxy = signal.coherence(analog_signal1.magnitude.flatten(), analog_signal2.magnitude.flatten(), fs=fs)
    return f, Cxy

# Perform Coherence Analysis
analog_signal2 = analog_signal  # Example with the same signal; replace with another signal
f_coherence, Cxy = coherence_analysis(analog_signal, analog_signal2)

# Plot Coherence
plt.figure(figsize=(10, 6))
plt.plot(f_coherence, Cxy)
plt.title('Coherence Analysis')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Coherence')
plt.show()

## Step 5: Phase-Amplitude Coupling (PAC) Analysis
Investigate the coupling between the phase of low-frequency oscillations and the amplitude of high-frequency oscillations.


In [ ]:
# PAC Analysis Module
def pac_analysis(analog_signal, low_freq=(4, 8), high_freq=(30, 100), fs=1000):
    """
    Investigate Phase-Amplitude Coupling (PAC) in LFP signals.
    
    Args:
    - analog_signal (neo.AnalogSignal): LFP data in Neo's AnalogSignal format.
    - low_freq (tuple): Low-frequency range for phase extraction.
    - high_freq (tuple): High-frequency range for amplitude extraction.
    - fs (int): Sampling frequency.
    
    Returns:
    - pac (float): Modulation index (MI) for PAC.
    """
    pac = ephase.phase_amplitude_coupling(analog_signal, low_freq, high_freq, fs=fs)
    return pac

# Perform PAC Analysis
pac = pac_analysis(analog_signal, low_freq=(4, 8), high_freq=(30, 100), fs=fs)
print(f'PAC Modulation Index: {pac}')

## Step 6: Cross-Frequency Coupling (CFC) Analysis
Measure interactions between oscillatory activities at different frequencies.

In [ ]:
# CFC Analysis Module
def cfc_analysis(analog_signal, fs=1000, phase_freqs=[(4, 8)], amplitude_freqs=[(30, 100)]):
    """
    Perform Cross-Frequency Coupling (CFC) analysis.
    
    Args:
    - analog_signal (neo.AnalogSignal): LFP data in Neo's AnalogSignal format.
    - fs (int): Sampling frequency.
    - phase_freqs (list of tuple): List of low-frequency ranges.
    - amplitude_freqs (list of tuple): List of high-frequency ranges.
    
    Returns:
    - cfc_matrix (np.ndarray): Matrix of CFC values.
    """
    # Implement the CFC analysis
    # Placeholder for the actual implementation
    cfc_matrix = np.random.rand(len(phase_freqs), len(amplitude_freqs))  # Replace with actual calculation
    return cfc_matrix

# Perform CFC Analysis
cfc_matrix = cfc_analysis(analog_signal, fs=fs)
print(f'CFC Matrix:\n{cfc_matrix}')

## Conclusion
In this notebook, we performed comprehensive Local Field Potential (LFP) analysis using Python and various scientific libraries. The steps included data loading, preprocessing, time-frequency analysis (STFT and Wavelet), coherence analysis, PAC, and CFC analysis. Each method offers a different insight into neural oscillations, rhythms, and interactions between brain regions.